In [1]:
from __future__ import print_function
import findspark
findspark.init()
from pyspark.sql import SparkSession, SQLContext, Row

C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
if __name__ == "__main__":

    # Create a SparkSession (Note, the config section is only for Windows!)
    spark = SparkSession.builder.master('local[*]').config("spark.driver.memory", "8g").config("spark.cores.max", "6").appName("FaultDetection").getOrCreate()
    #spark = SparkSession.builder.appName("RecommenderSystem").getOrCreate()
    
    # Load up data as dataframe
    data = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/My_Data/MS/CS657/Project/InputData/metadata_train.csv")
    data = data.limit(500)
    data.show(20)
    
    signalData = spark.read.option("header", "true").option("inferSchema", "true").parquet("C:/My_Data/MS/CS657/Project/InputData/train.parquet")
    signalData = signalData.limit(500)
    signalData.show(20)

+---------+--------------+-----+------+
|signal_id|id_measurement|phase|target|
+---------+--------------+-----+------+
|        0|             0|    0|     0|
|        1|             0|    1|     0|
|        2|             0|    2|     0|
|        3|             1|    0|     1|
|        4|             1|    1|     1|
|        5|             1|    2|     1|
|        6|             2|    0|     0|
|        7|             2|    1|     0|
|        8|             2|    2|     0|
|        9|             3|    0|     0|
|       10|             3|    1|     0|
|       11|             3|    2|     0|
|       12|             4|    0|     0|
|       13|             4|    1|     0|
|       14|             4|    2|     0|
|       15|             5|    0|     0|
|       16|             5|    1|     0|
|       17|             5|    2|     0|
|       18|             6|    0|     0|
|       19|             6|    1|     0|
+---------+--------------+-----+------+
only showing top 20 rows



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:50667)
Traceback (most recent call last):
  File "C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1b864252823>", line 14, in <module>
    signalData.show(20)
  File "c:\spark\python\pyspark\sql\dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "c:\spark\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:50667)
Traceback (most recent call last):
  File "C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1b864252823>", line 14, in <module>
    signalData.show(20)
  File "c:\spark\python\pyspark\sql\dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "c:\spark\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:50667)
Traceback (most recent call last):
  File "C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1b864252823>", line 14, in <module>
    signalData.show(20)
  File "c:\spark\python\pyspark\sql\dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "c:\spark\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:50667)
Traceback (most recent call last):
  File "C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1b864252823>", line 14, in <module>
    signalData.show(20)
  File "c:\spark\python\pyspark\sql\dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "c:\spark\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another ex

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:50667)
Traceback (most recent call last):
  File "C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1b864252823>", line 14, in <module>
    signalData.show(20)
  File "c:\spark\python\pyspark\sql\dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "c:\spark\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:50667)
Traceback (most recent call last):
  File "C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1b864252823>", line 14, in <module>
    signalData.show(20)
  File "c:\spark\python\pyspark\sql\dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "c:\spark\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:50667)
Traceback (most recent call last):
  File "C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1b864252823>", line 14, in <module>
    signalData.show(20)
  File "c:\spark\python\pyspark\sql\dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "c:\spark\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:50667)
Traceback (most recent call last):
  File "C:\Users\ashutosh.bansal\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1b864252823>", line 14, in <module>
    signalData.show(20)
  File "c:\spark\python\pyspark\sql\dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "c:\spark\python\pyspark\sql\utils.py", line 111, in deco
    return f(*a, **kw)
  File "c:\spark\python\lib\py4j-0.10.9-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another ex

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:50667)